In [7]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [100]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [17]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [12]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [13]:
# reflect an existing database into a new model

base= automap_base()

# reflect the tables

base.prepare(engine, reflect=True)

['measurement', 'station']

In [14]:
# View all of the classes that automap found
base.classes.keys()



['measurement', 'station']

In [15]:
# Save references to each table

measurement = base.classes.measurement

station = base.classes.station


In [16]:
# Create our session (link) from Python to the DB

session= Session(engine)


In [24]:
inspector= inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [42]:
measurement_columns= inspector.get_columns('measurement')
for column in measurement_columns:
    print(column["name"],column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [43]:
station_columns= inspector.get_columns('station')
for column in station_columns:
    print(column["name"],column["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


# Exploratory Precipitation Analysis

In [50]:
# Find the most recent date in the data set.

session.query(measurement.id, measurement.station, func.max(measurement.date), measurement.prcp, measurement.tobs).all()



[(2724, 'USC00519397', '2017-08-23', 0.0, 81.0)]

In [49]:
#confirm most recent date with a second method
session.query(measurement.date).order_by(measurement.date.desc()).first()

('2017-08-23',)

In [92]:
#save most recent date to a variable
soonest = session.query(measurement.id, measurement.station, func.max(measurement.date), measurement.prcp, measurement.tobs).all()
soonest_date= soonest[0][2]
soonest_date

'2017-08-23'

In [93]:
soonest_date= dt.datetime.strptime(soonest_date,'%Y-%m-%d')


In [107]:
measurement_columns= inspector.get_columns('measurement')
for column in measurement_columns:
    print(column["name"],column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [125]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 


# Calculate the date one year from the last date in data set.


year_ago= soonest_date - dt.timedelta(days=365)
year_ago= year_ago.strftime('%Y %m %d')

# Perform a query to retrieve the data and precipitation scores

results = session.query(measurement.id, measurement.station, measurement.date, measurement.prcp, measurement.tobs).filter(measurement.date >= year_ago).all()

# Save the query results as a Pandas DataFrame and set the index to the date column

    
temp_data= [{"Date": result[2], "Station": result[1], "Precipitation": result[3], "TOBs": result[4]} for result in results]

df= pd.DataFrame(temp_data).set_index("Date")
df

# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




,Station,Precipitation,TOBs
Date,,,
2016-01-01,USC00519397,0.00,62.0
2016-01-02,USC00519397,0.00,71.0
2016-01-03,USC00519397,0.00,63.0
2016-01-04,USC00519397,0.00,62.0
2016-01-05,USC00519397,0.00,68.0
...,...,...,...
2017-08-19,USC00516128,0.09,71.0
2017-08-20,USC00516128,NaN,78.0
2017-08-21,USC00516128,0.56,76.0


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [126]:
# Close Session
session.close()